In [1]:
from kafka import KafkaConsumer
from sqlalchemy import create_engine, text
import json
import logging

In [2]:
# Konfigurasi logging
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)

# --- KONEKSI KAFKA ---
try:
    consumer = KafkaConsumer(
        'clean_sensor_data', # Topik yang berisi data bersih
        bootstrap_servers='kafka:9092',
        auto_offset_reset='earliest', # Mulai membaca dari pesan paling awal
        group_id='db-writer-group', # ID grup untuk consumer ini
        value_deserializer=lambda x: json.loads(x.decode('utf-8')) # Decode JSON
    )
    log.info("Berhasil terhubung ke Kafka sebagai consumer.")
except Exception as e:
    log.error(f"Gagal terhubung ke Kafka: {e}")
    consumer = None

INFO:kafka.conn:<BrokerConnection client_id=kafka-python-2.2.15, node_id=bootstrap-0 host=kafka:9092 <connecting> [IPv4 ('172.18.0.7', 9092)]>: connecting to kafka:9092 [('172.18.0.7', 9092) IPv4]
INFO:kafka.conn:<BrokerConnection client_id=kafka-python-2.2.15, node_id=bootstrap-0 host=kafka:9092 <checking_api_versions_recv> [IPv4 ('172.18.0.7', 9092)]>: Broker version identified as 2.6
INFO:kafka.conn:<BrokerConnection client_id=kafka-python-2.2.15, node_id=bootstrap-0 host=kafka:9092 <connected> [IPv4 ('172.18.0.7', 9092)]>: Connection complete.
INFO:kafka.consumer.subscription_state:Updating subscribed topics to: ('clean_sensor_data',)
INFO:__main__:Berhasil terhubung ke Kafka sebagai consumer.


In [3]:
# --- KONEKSI DATABASE ---
DB_USER = 'user'
DB_PASSWORD = 'password'
DB_HOST = 'postgres_db-smart-manufacturing'
DB_PORT = '5432'
DB_NAME = 'machine_db'

In [4]:
try:
    engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')
    conn = engine.connect()
    log.info("Berhasil terhubung kembali ke database PostgreSQL.")
except Exception as e:
    log.error(f"Gagal terhubung ke database: {e}")
    conn = None

INFO:__main__:Berhasil terhubung kembali ke database PostgreSQL.


In [ ]:
# Loop untuk menulis data baru
try:
    log.info("Mulai menulis data sensor ke database...")
    for message in consumer:
        data = message.value
        # PERBAIKAN: Menambahkan kolom IMF ke query INSERT
        insert_query = text("""
            INSERT INTO sensor_readings (
                event_timestamp, machine_id, vibration, acoustic, 
                temperature, current, status, label,
                IMF_1, IMF_2, IMF_3
            ) VALUES (
                :event_timestamp, :machine_id, :vibration, :acoustic, 
                :temperature, :current, :status, :label,
                :IMF_1, :IMF_2, :IMF_3
            )
        """)
        conn.execute(insert_query, data)
        conn.commit()
        log.info(f"Data untuk machine_id {data['machine_id']} berhasil disimpan.")
except KeyboardInterrupt:
    log.warning("Proses dihentikan.")

INFO:__main__:Mulai menulis data sensor ke database...
INFO:kafka.conn:<BrokerConnection client_id=kafka-python-2.2.15, node_id=1 host=kafka:9092 <connecting> [IPv4 ('172.18.0.7', 9092)]>: connecting to kafka:9092 [('172.18.0.7', 9092) IPv4]
INFO:kafka.conn:<BrokerConnection client_id=kafka-python-2.2.15, node_id=1 host=kafka:9092 <connected> [IPv4 ('172.18.0.7', 9092)]>: Connection complete.
INFO:kafka.conn:<BrokerConnection client_id=kafka-python-2.2.15, node_id=bootstrap-0 host=kafka:9092 <connected> [IPv4 ('172.18.0.7', 9092)]>: Closing connection. 
INFO:kafka.cluster:Coordinator for group/db-writer-group is BrokerMetadata(nodeId='coordinator-1', host='kafka', port=9092, rack=None)
INFO:kafka.coordinator:Discovered coordinator coordinator-1 for group db-writer-group
INFO:kafka.coordinator.heartbeat:Starting new heartbeat thread
INFO:kafka.coordinator.consumer:Revoking previously assigned partitions set() for group db-writer-group
INFO:kafka.coordinator:Failed to join group db-write

In [ ]:
finally:
    conn.close()
    consumer.close()